In [1]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.table import Table
from astropy.time import Time
from astroquery.mast import Mast,Observations
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from glob import glob
import json

In [2]:
proposal_obs = Observations.query_criteria(obstype='all',obs_collection='JWST',instrument_name='NIRCAM/CORON')
#print("Number of observations:",len(proposal_obs))
print(proposal_obs)

intentType obs_collection provenance_name ... srcDen   obsid     objID  
---------- -------------- --------------- ... ------ --------- ---------
   science           JWST         CALJWST ...    nan  97925302 232275175
   science           JWST         CALJWST ...    nan  97925307 232275176
   science           JWST         CALJWST ...    nan  97925917 232275177
   science           JWST         CALJWST ...    nan  97925308 232275178
   science           JWST         CALJWST ...    nan 129690823 238300985
   science           JWST         CALJWST ...    nan 129687398 238300996
   science           JWST         CALJWST ...    nan 129687406 238301014
   science           JWST         CALJWST ...    nan 129687416 238301023
   science           JWST         CALJWST ...    nan 140300579 258159940
   science           JWST         CALJWST ...    nan 140300580 258159948
       ...            ...             ... ...    ...       ...       ...
   science           JWST         CALJWST ...    na

In [3]:

batch_size = 10
batches = [proposal_obs[i:i+batch_size] for i in range(0, len(proposal_obs), batch_size)]
t = [Observations.get_product_list(obs) for obs in batches]
products = unique(vstack(t), keys='productFilename')
filtered_products = Observations.filter_products(products)

In [4]:
prd_df = filtered_products.to_pandas()

In [5]:
prd_df['proposal_id'].unique()

array(['1075', '1184', '1189', '1193', '1194', '1386', '1411', '1412',
       '1441', '1482', '1536', '1537', '1538', '1550', '1563', '2278',
       '2540', '2780', '3840', '3989', '4050', '4451', '4452', '4454',
       '4497', '4498', '4558'], dtype=object)

In [6]:
df = prd_df.loc[prd_df['dataRights'] != 'EXCLUSIVE_ACCESS']

In [7]:
df['proposal_id'].unique()

array(['1075', '1184', '1189', '1193', '1194', '1386', '1411', '1412',
       '1441', '1482', '1536', '1537', '1538', '1550', '1563', '2278',
       '2780', '3840', '3989', '4050', '4451', '4452', '4454', '4497',
       '4498', '4558'], dtype=object)

In [8]:
df

,obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
18,83255958,JWST,image,jw01075001001_03104_00001_nrca2,source/target (L3) : association generator,S,mast:JWST/product/jw01075-o001_20231117t105138...,INFO,Minimum Recommended Products,ASN,NaN,CALJWST,1.10.1,1075,jw01075-o001_20231117t105138_image2_00048_asn....,1419,83255958,PUBLIC,2
39,83256032,JWST,image,jw01075002001_03104_00001_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01075-o002_20231117t105138...,INFO,Minimum Recommended Products,ASN,NaN,CALJWST,1.10.1,1075,jw01075-o002_20231117t105138_image2_00051_asn....,1419,83256032,PUBLIC,2
44,83256069,JWST,image,jw01075003001_03104_00001_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01075-o003_20231117t105138...,INFO,Minimum Recommended Products,ASN,NaN,CALJWST,1.10.1,1075,jw01075-o003_20231117t105138_image2_00011_asn....,1419,83256069,PUBLIC,2
65,83256121,JWST,image,jw01075005001_03104_00001_nrcalong,source/target (L3) : association generator,S,mast:JWST/product/jw01075-o005_20231117t105138...,INFO,Minimum Recommended Products,ASN,NaN,CALJWST,1.10.1,1075,jw01075-o005_20231117t105138_image2_00021_asn....,1425,83256121,PUBLIC,2
70,83337070,JWST,image,jw01075006001_03104_00001_nrcalong,source/target (L3) : association generator,S,mast:JWST/product/jw01075-o006_20231117t105138...,INFO,Minimum Recommended Products,ASN,NaN,CALJWST,1.10.1,1075,jw01075-o006_20231117t105138_image2_00005_asn....,1425,83337070,PUBLIC,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48242,203108931,JWST,image,jw04558009001_02102_00001_nrcalong,FGS guide star tracking,S,mast:JWST/product/jw04558009001_gs-track_20240...,AUXILIARY,NaN,GS-TRACK,NaN,CALJWST,NaN,4558,jw04558009001_gs-track_2024014204224_stream.fits,584640,203116813,PUBLIC,1
48243,203108931,JWST,image,jw04558009001_02102_00001_nrcalong,FGS guide star tracking,S,mast:JWST/product/jw04558009001_gs-track_20240...,AUXILIARY,NaN,GS-TRACK,NaN,CALJWST,NaN,4558,jw04558009001_gs-track_2024014204224_uncal.fits,702720,203116813,PUBLIC,1
48244,183369444,JWST,image,jw04558-c1002_t001_nircam_f200w-maskrnd-sub320...,source/target (L3) : association pool,D,mast:JWST/product/jw04558_20231012t201828_pool...,INFO,Minimum Recommended Products,POOL,NaN,CALJWST,1.11.4,4558,jw04558_20231012t201828_pool.csv,37133,183369444,PUBLIC,3
48245,183254640,JWST,image,jw04558001001_02102_00001_nrcalong,source/target (L3) : association pool,S,mast:JWST/product/jw04558_20231201t195253_pool...,INFO,NaN,POOL,NaN,CALJWST,1.11.4,4558,jw04558_20231201t195253_pool.csv,73745,183313487,PUBLIC,2


In [ ]:
def save_products(df,programs):

    for program in programs:
        
        jw = df[(df['proposal_id']==program)  & (df['productType'] == 'SCIENCE') & (df['calib_level'] == 2) & (df['productSubGroupDescription'] == 'RATEINTS')]
        products = Table.from_pandas(jw)
        data_dir = f"/data/scratch/sarperyurtseven/dataset/NIRCAM/{program}"
        os.makedirs(data_dir,exist_ok=True)
        Observations.download_products(products,mrp_only=False,download_dir=f'')
        print(f'{program} is done.')


In [ ]:
programs = ['1075', '1184', '1189', '1193', '1194', '1386', '1411', '1412',
       '1441', '1482', '1550', '1563', '2278',
       '2780', '3840', '3989', '4050', '4451', '4452', '4454', '4497',
       '4498', '4558']

In [ ]:
save_products(df=df,programs=programs)